# Task 1: Load the Data (1 Mark)


Download the fruit dataset from Moodle and use Pandas to read it into Python as a Dataframe.

In [1]:
import pandas as pd
import seaborn as sns

fruit = pd.read_table('/Users/lukasbenic/Desktop/Bath Uni/2nd Year/Term 2/ML/fruit_data_with_colors.txt');
fruit.head(20)

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79
5,2,mandarin,mandarin,80,5.8,4.3,0.77
6,2,mandarin,mandarin,80,5.9,4.3,0.81
7,2,mandarin,mandarin,76,5.8,4.0,0.81
8,1,apple,braeburn,178,7.1,7.8,0.92
9,1,apple,braeburn,172,7.4,7.0,0.89


# Task 2:  Nearest Neighbours (3 Marks)

1. Extract the mass, width, height and color_score data from the test set as variable X and fruit_name as y.
2. Use train_test_split from sklearn to divide the data into training and test data with a 70/30 split.
3. Scale the data using StandardScaler from sklearn.preprocessing
4. Loop over the range 1 to 8 building a kNN classifier using KNeighboursClassifier from sklearn displaying the
confusion matrix and accuracy score for each with the test data.

The iris example notebook from the first lecture is available on Moodle as a guide to completing this task.

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 


In [32]:
fruit_vector = fruit[["width","height","color_score","fruit_name"]]
X = fruit_vector.iloc[:,:-1].values
y = fruit_vector.iloc[:, -1].values
print(X[0])
print(y[0:,])


[8.4  7.3  0.55]
['apple' 'apple' 'apple' 'mandarin' 'mandarin' 'mandarin' 'mandarin'
 'mandarin' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'orange' 'orange' 'orange' 'orange' 'orange' 'orange' 'orange' 'orange'
 'orange' 'orange' 'orange' 'orange' 'orange' 'orange' 'orange' 'orange'
 'orange' 'orange' 'orange' 'lemon' 'lemon' 'lemon' 'lemon' 'lemon'
 'lemon' 'lemon' 'lemon' 'lemon' 'lemon' 'lemon' 'lemon' 'lemon' 'lemon'
 'lemon' 'lemon']


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=3421)
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)


print(X_test[0, :])

print(y_train[0])

[ 0.44209986 -0.29153433 -1.00325029]
apple


In [34]:
classifier = KNeighborsClassifier(n_neighbors=8)  
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("kNN %):", metrics.accuracy_score(y_test, y_pred)*100)

[[4 0 0 3]
 [0 8 0 0]
 [0 0 0 2]
 [0 0 0 3]]
kNN %): 75.0


## Task 3: Library Gaussian Naïve Bayes (1 Mark)

In this task you will replace the kNN classifier with a Gaussian Naïve Bayes classifier using the GaussianNB from
sklearn.naive_bayes. You will only need one classifier. Again, you should display the confusion matrix and accuracy
score for the test data.

In [35]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred2 = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred2))
print("kNN %):", metrics.accuracy_score(y_test, y_pred2)*100)

[[7 0 0 0]
 [0 8 0 0]
 [0 0 2 0]
 [2 0 0 1]]
kNN %): 90.0


# Task 4: Hand-coded Gaussian Naïve Bayes (5 Marks)

In this task you will replace the GaussianNB from sklearn.naive_bayes with a version you implement in python. You
are not required to build a class, functions are sufficient.
Gaussian Naïve Bayes is a Maximum A Posteriori (MAP) approach. It finds the maximum value of P(c|d) where d is
the input vector and c is the class.

In [42]:
import collections
import copy

class GNB:

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        self._classes = np.unique(y_train)
        n_c = np.size(self._classes)
        self._priors = np.zeros(n_c, dtype=np.float64)
        self._mean = np.zeros((n_c, X_train.shape[1]), dtype=np.float64)
        self._variance = np.zeros((n_c, X_train.shape[1]), dtype=np.float64)

        self._mean_prior(X_train, y_train)
        self._var(y_train)

    def _mean_prior(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        y_dict = collections.Counter(y_train)
        i = 0
        for index, c in enumerate(self._classes):
            self._priors[index] = np.float64(y_dict[c]) / y_train.shape[0]

            for y in range(0, np.size(y_train)):
                if self._classes[index] == y_train[y]:
                    i += 1
                    self._mean[index, :] += X_train[y]

            self._mean[index, :] = self._mean[index, :] / i

    def _var(self, y_train: np.ndarray) -> None:
        i = 0
        for index, c in enumerate(self._classes):

            for y in range(0, np.size(y_train)):
                if self._classes[index] == y_train[y]:
                    i += 1
                    mean_copy = copy.deepcopy(self._mean)
                    self._variance[index, :] += (abs(X_train[y] - mean_copy[index, :]) ** 2)

            self._variance[index, :] = (self._variance[index, :] / i)

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        return np.array([self._predict(d_test) for d_test in X_test])

    def _predict(self, d_test: np.float64) -> str:
        """
        """
        posteriors = []

        for index, c in enumerate(self._classes):
            prior = np.log(self._priors[index])

            pdf_var = self._pdf(index, d_test)
            pdf_var[pdf_var == 0.] = np.float64(2**-10)  # Setting zero values to value of negligible size

            class_cond = np.sum(np.log(pdf_var))
            posteriors.append(prior + class_cond)

        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_index : int, d_test: np.ndarray) -> np.ndarray:
        return np.array((np.exp(- (d_test - self._mean[class_index]) ** 2 / (2 * self._variance[class_index]))) / (np.sqrt(2 * np.pi * self._variance[class_index])))

In [43]:
clf2 = GNB()
clf2.fit(X_train, y_train)
y_pred3 = clf2.predict(X_test)
print(confusion_matrix(y_test, y_pred3))
print("kNN %):", metrics.accuracy_score(y_test, y_pred3)*100)

[[7 0 0 0]
 [0 8 0 0]
 [0 0 2 0]
 [1 0 0 2]]
kNN %): 95.0
